In [1]:
#Importar librerías para tratamiento de datos
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
import re
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

from selenium.common.exceptions import NoSuchElementException

In [2]:
#Creamos la función

def get_data(driver,xpath):
    try:
        data = driver.find_element("xpath", xpath).text
    except:
        data = "N/A"
    return data


def extraer_datos_actores(nombre_archivo):
    df_pelis = pd.read_csv("peliculas_api.csv")

    df_pelis["ID de la Película"] = df_pelis["ID de la Película"].astype(str)

    driver = webdriver.Chrome()
    driver.maximize_window()

    for id_imdb in df_pelis["ID de la Película"][:5]:
        try:
            url = f"https://www.imdb.com/title/{id_imdb}/"
            driver.get(url)

            sleep(2)
            try:
                driver.find_element("xpath", '//*[@id="__next"]/div/div/div[2]/div/button[2]').click()
            except NoSuchElementException:
                pass 

            sleep(2) #verificar si este sleep es util o no
            driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/ul/li[1]/a').click()
    
            sleep(2)

            lista_actores = []
            #lista_actores= driver.find_element("class name", "cast list")[0].find_elements("tag name", "tr")
            #numero_maximo_actores = 10 if len(lista_actores)-1 > 10 else len(lista_actores)-1

            nombre_peli = get_data(driver,'//*[@id="main"]/div[1]/div[1]/div/div[1]/h3/a')
            
            for i in range(2,12):
                try:

                    sleep(2)
                    driver.find_element("xpath",f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()  
                    sleep(3)

                    nombre_actxr = driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/h1/span').text
                    conocido_por1 = driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[2]/section[1]/div[2]/div/div[2]/div[1]/div[2]/div[1]/a').text
                    try:
                        conocido_por2 = driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[2]/section[1]/div[2]/div/div[2]/div[2]/div[2]/div[1]/a').text
                    except:
                        conocido_por2 = "N/A"    
                    conocido_por = [conocido_por1, conocido_por2] if conocido_por2 != "N/A" else [conocido_por1]
                
                    try:
                        fecha_nacimiento = driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div/div[2]/section/aside/div/span[2]').text
                    except:
                        fecha_nacimiento = "N/A"
                    year_nacimiento = fecha_nacimiento.split(" ")[-1] if fecha_nacimiento.split(" ")[-1].isdigit() else "N/A"

                    try:
                        profesiones = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text.split("\n")
                    except:
                        profesiones = "N/A"
                    profesiones_string = ", ".join(profesiones) if profesiones != "N/A" else "N/A"

                    try:
                        premios = driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul/li/span').text
                    except:
                        premios = "N/A"

                    info_actores = (nombre_peli,nombre_actxr,conocido_por,year_nacimiento,profesiones_string,premios)
                    lista_actores.append(info_actores)

                except NoSuchElementException:
                    print(f"\033[95mNo se encontró el actor número {i-1}\033[0m")
                    pass

                finally:
                    driver.back()
            
            print("Datos de los actores:")
            print(lista_actores)

        except Exception as e:
            print(f"Error al procesor la película con ID de IMDb {id_imdb}: {e}")

    driver.quit()

In [3]:
extraer_datos_actores("peliculas_api.csv")

Datos de los actores:
[('Pál Adrienn', 'Éva Gábor', ['Europa, East', 'Pál Adrienn'], 'N/A', 'Reparto', 'N/A'), ('Pál Adrienn', 'István Znamenák', ['Az állampolgár', 'Taxidermia'], '1966', 'Reparto, Dirección, Dirección de fotografía y cámara', 'N/A'), ('Pál Adrienn', 'Ákos Horváth', ['Kontroll', 'Haunting Trophies'], '1968', 'Reparto, Guion', 'N/A'), ('Pál Adrienn', 'Lia Pokorny', ['Ízig-vérig', 'Well'], '1971', 'Reparto', 'N/A'), ('Pál Adrienn', 'Izabella Hegyi', ['Friss levegö', 'Pál Adrienn'], 'N/A', 'Reparto', 'N/A'), ('Pál Adrienn', 'Szabolcs Thuróczy', ['Matones sobre ruedas', 'Sandwich'], '1971', 'Reparto, Guion', '3 premios'), ('Pál Adrienn', 'Ági Margitai', ['Ilusiones de un mentiroso', 'Ripacsok'], '1937', 'Reparto', 'N/A'), ('Pál Adrienn', 'Eszter Márton', ['Nunca moriremos (Sose halunk meg)', 'Jóban Rosszban'], '1969', 'Guion, Reparto', 'N/A'), ('Pál Adrienn', 'Lóránt Matta', ['Pál Adrienn', 'Mi kérünk elnézést!'], '1973', 'Reparto', 'N/A'), ('Pál Adrienn', 'Róbert Kardos',

KeyboardInterrupt: 

In [10]:
#con el class y list

def extraer_datos_actores(nombre_archivo):
    # He cambiado el string por la variable nombre_archivo
    df_pelis = pd.read_csv(nombre_archivo)

    df_pelis["ID de la Película"] = df_pelis["ID de la Película"].astype(str)
    datos_actores_total = []
    
    driver = webdriver.Chrome()
    driver.maximize_window()

    for id_imdb in df_pelis["ID de la Película"][:6]:
        try:
            url = f"https://www.imdb.com/title/{id_imdb}/"
            driver.get(url)

            sleep(2)
            try:
                driver.find_element(
                    "xpath", '//*[@id="__next"]/div/div/div[2]/div/button[2]').click()
            except NoSuchElementException:
                pass

            sleep(2)  # verificar si este sleep es util o no
            driver.find_element(
                "xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/ul/li[1]/a').click()

            sleep(2)

            lista_actores = []

            # Busco todos los actores de la película
            actores = driver.find_elements(
                'class name', 'cast_list')[0].find_elements('tag name', 'tr')
            
            # aquí cuento cuantos actores hay en la lista. Le resto 1 porque el primer elemento es el encabezado de la tabla y no tiene datos
            numero_maximo_actores = 10 if len(
                actores)-1 > 10 else len(actores)-1

            nombre_peli = get_data(
                driver, '//*[@id="main"]/div[1]/div[1]/div/div[1]/h3/a')

            for i in range(2, numero_maximo_actores):
                try:

                    sleep(2)

                    driver.find_element(
                        "xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
                    sleep(3)

                    nombre_actxr = get_data(
                        # Cambiado por la función
                        driver, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/h1/span')

                    # Cambiado por la función
                    conocido_por1 = get_data(
                        driver, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[2]/section[1]/div[2]/div/div[2]/div[1]/div[2]/div[1]/a')

                    # Cambiado por la función
                    conocido_por2 = get_data(
                        driver, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[2]/section[1]/div[2]/div/div[2]/div[2]/div[2]/div[1]/a')
                    conocido_por = [
                        conocido_por1, conocido_por2] if conocido_por2 != "N/A" else [conocido_por1]

                    # Cambiado por la función
                    fecha_nacimiento = get_data(
                        driver, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div/div[2]/section/aside/div/span[2]')
                    year_nacimiento = fecha_nacimiento.split(
                        " ")[-1:][0] if fecha_nacimiento != "N/A" else "N/A"

                    # Cambiado por la función
                    profesiones = get_data(
                        driver, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul')
                    profesiones_string = ", ".join(
                        profesiones.split("\n")) if profesiones != "N/A" else "N/A"

                    # Cambiado por la función
                    premios = get_data(
                        driver, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul/li/span')

                    info_actores = (nombre_peli, nombre_actxr, conocido_por,
                                    year_nacimiento, profesiones_string, premios)
                    lista_actores.append(info_actores)
                    

                except NoSuchElementException:
                    print(f"\033[95mNo se encontró el actor número {i-1}\033[0m")
                    pass

                finally:
                    driver.back()

            print("Datos de los actores:")
            print(lista_actores)
            
            datos_actores_total.extend(lista_actores)
        except Exception as e:
            print(f"Error al procesor la película {nombre_peli} con ID de IMDb {id_imdb}: {e}")

    driver.quit()
    return datos_actores_total


In [11]:
df_peliculas = pd.read_csv("peliculas_api.csv")

In [12]:
# Llamamos a  la función con nuestro archivo de pelis que sacamos de la API.
extraccion = extraer_datos_actores("peliculas_api.csv")

Error al procesor la película Pál Adrienn con ID de IMDb tt0146592: Message: element click intercepted: Element <a href="/name/nm3140529/?ref_=ttfc_fc_cl_t1">...</a> is not clickable at point (198, 516). Other element would receive the click: <ul class="ipc-list searchCatSelector ipc-list--baseAlt" role="menu" aria-orientation="vertical">...</ul>
  (Session info: chrome=124.0.6367.203)
Stacktrace:
	GetHandleVerifier [0x00007FF708DD1522+60802]
	(No symbol) [0x00007FF708D4AC22]
	(No symbol) [0x00007FF708C07CE4]
	(No symbol) [0x00007FF708C5E877]
	(No symbol) [0x00007FF708C5C344]
	(No symbol) [0x00007FF708C5980A]
	(No symbol) [0x00007FF708C589E2]
	(No symbol) [0x00007FF708C4AE8B]
	(No symbol) [0x00007FF708C7AB7A]
	(No symbol) [0x00007FF708C4A7C6]
	(No symbol) [0x00007FF708C7AD90]
	(No symbol) [0x00007FF708C9A224]
	(No symbol) [0x00007FF708C7A923]
	(No symbol) [0x00007FF708C48FEC]
	(No symbol) [0x00007FF708C49C21]
	GetHandleVerifier [0x00007FF7090D41BD+3217949]
	GetHandleVerifier [0x00007FF

In [13]:
df_extraccion = df = pd.DataFrame(extraccion, columns=["nombre_peli", "nombre_actxr", "conocido_por", "year_nacimiento", "profesiones_string", "premios"])



In [14]:
df_extraccion

,nombre_peli,nombre_actxr,conocido_por,year_nacimiento,profesiones_string,premios
0,So Much for Justice!,Kornél Mundruczó,"[Jupiter's Moon, White God]",1975,"Reparto, Dirección, Guion",46 premios y 35 nominaciones
1,So Much for Justice!,Daniel Olbrychski,"[Salt, El diluvio]",1945,"Reparto, Dirección de la segunda unidad o asis...",8 premios y 2 nominaciones
2,So Much for Justice!,György Cserhalmi,"[Kontroll, Zelary]",1948,Reparto,8 premios y 2 nominaciones
3,So Much for Justice!,Gábor Nagypál,"[Inferno, Monte Carlo]",1975,Reparto,Gabor Nagypal
4,So Much for Justice!,László Gálffi,"[Wagner, Liszt Ferenc]",1952,Reparto,László Gálffy
5,So Much for Justice!,Zoltán Mucsi,"[El topo, 1]",1957,Reparto,3 premios
6,So Much for Justice!,Florian Teichtmeister,"[Sommer der Gaukler, Polly Adler]",1979,"Reparto, Banda sonora","1,73 m"
7,So Much for Justice!,Miklós Székely B.,"[Sátántangó, Hintaszék]",1948,Reparto,4 premios y 1 nominación
